In [1]:
import tensorflow as tf
#import tensorflow.contrib.learn.python.learn as learn
import tflearn 
import scipy as sp
import numpy as np 
import matplotlib.pyplot as plt
import six
from sklearn.metrics import roc_curve
import datetime
%matplotlib inline

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
"""ecgData = np.load("/media/john/SAMSUNG/CNN/3D-conv/ecgData.npy")
ecgClass = np.load("/media/john/SAMSUNG/CNN/3D-conv/ecgClass.npy")
unseenData = np.load("/media/john/SAMSUNG/CNN/3D-conv/unseenData.npy")
unseenClass = np.load("/media/john/SAMSUNG/CNN/3D-conv/unseenClass.npy")

comData = np.concatenate((ecgData, unseenData))
comClass = np.concatenate((ecgClass, unseenClass))

np.save("/media/john/SAMSUNG/CNN/3D-conv/comData.npy", comData)
np.save("/media/john/SAMSUNG/CNN/3D-conv/comClass.npy", comClass)"""

'ecgData = np.load("/media/john/SAMSUNG/CNN/3D-conv/ecgData.npy")\necgClass = np.load("/media/john/SAMSUNG/CNN/3D-conv/ecgClass.npy")\nunseenData = np.load("/media/john/SAMSUNG/CNN/3D-conv/unseenData.npy")\nunseenClass = np.load("/media/john/SAMSUNG/CNN/3D-conv/unseenClass.npy")\n\ncomData = np.concatenate((ecgData, unseenData))\ncomClass = np.concatenate((ecgClass, unseenClass))\n\nnp.save("/media/john/SAMSUNG/CNN/3D-conv/comData.npy", comData)\nnp.save("/media/john/SAMSUNG/CNN/3D-conv/comClass.npy", comClass)'

In [2]:
comData = np.load("/media/john/SAMSUNG/CNN/3D-conv/comData.npy")
comClass = np.load("/media/john/SAMSUNG/CNN/3D-conv/comClass.npy")

In [3]:
def processClassData(classData):
    """
    Process classData.
    Returns a one-hot array of shape [len(classData), 2].
    """
    # Convert label data to one-hot array
          
    classDataOH = np.zeros((len(classData),2))
    classDataOH[np.arange(len(classData)), classData] = 1
    
    return classDataOH

comData = comData[..., np.newaxis]
comClassOH = processClassData(comClass)

In [4]:
# kfold x-validation...
k = 3
kfoldData = np.array_split(comData, k)
kfoldLabelsOH = np.array_split(comClassOH, k)
kfoldLabels = np.array_split(comClass, k)
print(kfoldData[0].shape)

(134, 2000, 19, 17, 1)


# Neural Network

In [5]:
healthEval = []
illEval = []
spec = []
sens = []
unseenSpec = []
unseenSens = []
unseenAvg = []
roc = []

In [6]:
subsamp = 40
numepoch = 2

In [7]:
perf = {}
perf['in_acc'] = np.zeros((k, numepoch))
perf['out_acc'] = np.zeros((k, numepoch))
perf['P_acc'] = np.zeros((k, numepoch))
perf['H_acc'] = np.zeros((k, numepoch))
perf['ROC'] = []

In [8]:
for i in np.arange(0,k,1):
    perf['ROC'].append([])
    print 'K= ', i
    sess = tf.InteractiveSession()
    tf.reset_default_graph()
    tflearn.initializations.normal()

    # Input layer:
    net = tflearn.layers.core.input_data(shape=[None, 50, 19, 17, 1])

    # First layer:
    net = tflearn.layers.conv.conv_3d(net, 8, [5,5,5],  activation="leaky_relu", 
                                      regularizer='L2', weight_decay=0.001)
    net = tflearn.layers.conv.max_pool_3d(net, 2, strides=2)

    # Second layer:
    net = tflearn.layers.conv.conv_3d(net, 16, [5,5,5], activation="leaky_relu", 
                                      regularizer='L2', weight_decay=0.001)
    net = tflearn.layers.conv.max_pool_3d(net, 2, strides=2)

    # Fully connected layer
    net = tflearn.layers.core.fully_connected(net, 1024, regularizer="L2",
                                              weight_decay=0.001, activation="leaky_relu")
    net = tflearn.layers.core.fully_connected(net, 1024, regularizer="L2",
                                              weight_decay=0.001, activation="leaky_relu")
    
    # Dropout layer:
    net = tflearn.layers.core.dropout(net, keep_prob=0.5)

    # Output layer:
    net = tflearn.layers.core.fully_connected(net, 2, activation="softmax")

    net = tflearn.layers.estimator.regression(net, optimizer='adam', 
                                              learning_rate=0.0001, loss='categorical_crossentropy')
    #Could use loss function: 'roc_auc_score'
    
    model = tflearn.DNN(net, tensorboard_verbose=3)
    
    dummyData = np.reshape(np.concatenate(kfoldData[:i] + kfoldData[i+1:], axis=0), [-1, 2000, 19, 17, 1])
    dummyData = dummyData[:,::subsamp]
    dummyLabels = np.reshape(np.concatenate(kfoldLabelsOH[:i] + kfoldLabelsOH[i+1:], axis=0), [-1, 2])
    
    illTest = []
    healthTest = []
    for index, item in enumerate(kfoldLabels[i]):
        if item == 1:
            illTest.append(kfoldData[i][index])
        if item == 0:
            healthTest.append(kfoldData[i][index])
    healthLabel = np.tile([1,0], (len(healthTest), 1))
    illLabel = np.tile([0,1], (len(illTest), 1))
    
    for j in range(numepoch):
        model.fit(dummyData, dummyLabels, batch_size=8, n_epoch=1, show_metric=True)
        perf['in_acc'][i][j] = model.evaluate(dummyData, dummyLabels)[0]
        perf['out_acc'][i][j] = model.evaluate(kfoldData[i][:,::subsamp], kfoldLabelsOH[i])[0]
        perf['P_acc'][i][j] = model.evaluate(np.array(illTest)[:,::subsamp], illLabel)[0]
        perf['H_acc'][i][j] = model.evaluate(np.array(healthTest)[:,::subsamp], healthLabel)[0]
        predicted = np.array(model.predict(np.array(kfoldData[i])[:,::subsamp]))
        fpr, tpr, th = roc_curve(kfoldLabels[i], predicted[:,1])
        perf['ROC'][i].append([fpr, tpr])

    sens.append(model.evaluate(np.array(healthTest)[:,::subsamp], healthLabel))
    spec.append(model.evaluate(np.array(illTest)[:,::subsamp], illLabel))

    # Get roc curve data
    predicted = np.array(model.predict(np.array(kfoldData[i])[:,::subsamp]))
    fpr, tpr, th = roc_curve(kfoldLabels[i], predicted[:,1])
    roc.append([fpr, tpr])

Training Step: 67  | total loss: 0.60912 | time: 0.327s
| Adam | epoch: 002 | loss: 0.60912 - acc: 0.6611 -- iter: 264/267
Training Step: 68  | total loss: 0.61899 | time: 0.335s
| Adam | epoch: 002 | loss: 0.61899 - acc: 0.6420 -- iter: 267/267
--


In [ ]:
#model.save("./outData/models/3d_conv8-16-50.tflearn")
#model.load("./outData/models/3d_conv.tflearn")

In [ ]:
print("Specificity:", spec, "\nAvg:", np.mean(spec), "\nSensitivity:", sens, "\nAvg:", np.mean(sens))
for i in np.arange(k):
    plt.plot(roc[i][0], roc[i][1])

## Hyperparameter Optimisation

We could potentially optimise:
    * Regression layer learning rate
    * Filter shape
    * Pooling shape and stride
    * Weight decay of regulariser (in Conv layer & Fully connected layers)
    * Subsamp